In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import os.path as path
import random
from IPython.display import HTML

import exdir
import numpy as np
import pandas as pd
from tqdm import tqdm
import skvideo
import yaml

from cow_tus.analysis.visualizations.viewer import play
import cow_tus.data.transforms as transforms

os.chdir('/home/gangus/cow-tus')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
raw_dir = "/data4/cow-tus-data/raw"
out_dir = "/data4/cow-tus-data/processed"

hypothesis_conditions = ['single-instance-learning', 'temporal-downsample']
group_dir = path.join('data', *hypothesis_conditions)

In [40]:
f = exdir.File('/data4/cow-tus-data/processed.exdir')
root_group = f['/'.join(hypothesis_conditions)]

In [47]:
columns = pd.read_csv(path.join(raw_dir, 'labels_instance_level.csv')).keys()
raw_instance_level_df = pd.read_excel(path.join(raw_dir, 'labels_instance_full.xlsx'))
raw_instance_level_df.columns = columns
raw_instance_level_df.to_csv(path.join(raw_dir, 'labels_instance_full.csv'), index=False)

In [42]:
notes_groups = raw_instance_level_df.groupby('notes')
for note, note_group in notes_groups:
    print(note, list(note_group['id']))

File L77 should be L7 ['GN4487']
I did not rate the file named RV4 ['GN4521']
I only have 1 clip here - did I miss something in the copy? ['CH4737', 'CH4838']
Is this file missing? ['OR4205', 'WT4493']
Missing L6 ['WT5167', 'OR4691']
Missing L7 ['TN5025']
Missing R5 ['OR5175']
Missing an R4 ['GY4168']
Notice mistake in naming videofile. R44 should be R4 ['TN4236']
Notice that one file ending on L, should be named L5 ['TN4307']
Notice that one file ending on L, should be named LV ['TQ4042']
R6 is named R66 ['CH4732']
Sickest cow I have ever seen ['GN4867']
Their labeling of 3 is actually incorrect ['PK4450']
Use L5 2nd, missing RV ['TN5021']
Use L6 (not L6.2 - my computer says that file is broken) ['WT4485']
Use R5.3. Also R6 incorrectly called R66 ['CH4837']
Use the R6REAL ['TN3875']
Use the the RV (not the RV2ND) ['CH4307']


In [43]:
list(root_group['TN5021'].keys())

['5021L5',
 '5021L5.2ND',
 '5021L6',
 '5021L7',
 '5021LV',
 '5021R4',
 '5021R5',
 '5021R6']

In [44]:
def update_group(exam_id, old_loop_id, new_loop_id):
    root_group[exam_id][new_loop_id] = root_group[exam_id][old_loop_id]
    root_group[exam_id][new_loop_id].attrs.update(root_group[exam_id][old_loop_id].attrs)
    root_group[exam_id][old_loop_id].attrs.update({
        'exdir.corrupted_id': True
    })

In [45]:
update_group('TN5021', '5021L5.2ND', '5021L5')

In [54]:
update_group('GN4487', '44887RV', '4487RV')

In [55]:
update_group('GN4487', '487L6', '4487L6')

In [56]:
update_group('GN4487', '487L77', '4487L7')

In [57]:
list(root_group['GN4521'].keys())

['4521L5',
 '4521L6',
 '4521L7',
 '4521LV',
 '4521R4',
 '4521R5',
 '4521R6',
 '4521RV4',
 '4521VRV']

In [58]:
update_group(exam_id='GN4521', old_loop_id='4521VRV', new_loop_id='4521RV')

In [59]:
update_group(exam_id='GN4521', old_loop_id='4521RV4', new_loop_id='4521RV4')

In [63]:
list(root_group['TN4236'].keys())

['4236L5',
 '4236L6',
 '4236L7',
 '4236LV',
 '4236R4',
 '4236R44',
 '4236R5',
 '4236R6',
 '4236RV']

In [62]:
update_group(exam_id='TN4236', old_loop_id='4236R44', new_loop_id='4236R4')

In [64]:
list(root_group['TN4307'].keys())

['4307L', '4307L6', '4307L7', '4307LV', '4307R4', '4307R5', '4307R6', '4307RV']

In [65]:
update_group(exam_id='TN4307', old_loop_id='4307L', new_loop_id='4307L5')

In [66]:
list(root_group['TQ4042'].keys())

['4042L', '4042L5', '4042L6', '4042R4', '4042R5', '4042R6', '4042RV', '4043L7']

In [67]:
update_group(exam_id='TQ4042', old_loop_id='4042L', new_loop_id='4042LV')

In [92]:
attrs_df = pd.read_csv('data/single-instance-learning/temporal-downsample/attrs.csv', index_col=0)

In [90]:
# loop_paths = attrs_df['exdir.loop_path']
# exams = {}
# for loop_path in tqdm(loop_paths):
#     with open(path.join(loop_path, 'attributes.yaml'), 'r') as f:
#         try:
#             d = yaml.safe_load(f)
#             if 'processed.corrupted_id' in d:
#                 del d['processed.corrupted_id']
#                 d['exdir.corrupted_id'] = True
#                 exams[d['exdir.loop_path']] = d
#         except yaml.YAMLError as exc:
#             print(exc)
# for loop_path, d in tqdm(exams.items()):
#     with open(path.join(loop_path, 'attributes.yaml'), 'w') as f:
#         f.write(yaml.dump(d))

100%|██████████| 9/9 [00:00<00:00, 159.54it/s]


In [93]:
corrupted_groups = attrs_df.groupby('processed.corrupted_id')
for corrupt, corrupted_group in corrupted_groups:
    for idx, corrupted_loop in corrupted_group.iterrows():
        print(corrupted_loop['exdir.loop_path'])

/data/cow-tus-data/processed.exdir/single-instance-learning/temporal-downsample/TN4236/4236R44
/data/cow-tus-data/processed.exdir/single-instance-learning/temporal-downsample/TQ4042/4042L
/data/cow-tus-data/processed.exdir/single-instance-learning/temporal-downsample/TN4307/4307L
/data/cow-tus-data/processed.exdir/single-instance-learning/temporal-downsample/GN4521/4521RV4
/data/cow-tus-data/processed.exdir/single-instance-learning/temporal-downsample/GN4521/4521VRV
/data/cow-tus-data/processed.exdir/single-instance-learning/temporal-downsample/GN4487/44787L5
/data/cow-tus-data/processed.exdir/single-instance-learning/temporal-downsample/GN4487/44887RV
/data/cow-tus-data/processed.exdir/single-instance-learning/temporal-downsample/GN4487/487L6
/data/cow-tus-data/processed.exdir/single-instance-learning/temporal-downsample/GN4487/487L77
